Regresión - Experimentación con modelos adicionales

En el cuaderno anterior, utilizamos modelos de regresión simples para observar la relación entre las características de un conjunto de datos de alquiler de bicicletas. En este cuaderno, experimentaremos con modelos más complejos para mejorar el rendimiento de nuestra regresión.

Empezaremos cargando los datos de alquiler de bicicletas como un Pandas DataFrame y visualizando las primeras filas. También dividiremos nuestros datos en conjuntos de datos de entrenamiento y de prueba.

In [ ]:
# Import modules we'll need for this notebook
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# load the training dataset
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/ml-basics/daily-bike-share.csv
bike_data = pd.read_csv('daily-bike-share.csv')
bike_data['day'] = pd.DatetimeIndex(bike_data['dteday']).day
numeric_features = ['temp', 'atemp', 'hum', 'windspeed']
categorical_features = ['season','mnth','holiday','weekday','workingday','weathersit', 'day']
bike_data[numeric_features + ['rentals']].describe()
print(bike_data.head())


# Separate features and labels
# After separating the dataset, we now have numpy arrays named **X** containing the features, and **y** containing the labels.
X, y = bike_data[['season','mnth', 'holiday','weekday','workingday','weathersit','temp', 'atemp', 'hum', 'windspeed']].values, bike_data['rentals'].values

# Split data 70%-30% into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print ('Training Set: %d rows\nTest Set: %d rows' % (X_train.shape[0], X_test.shape[0]))

Ahora tenemos los siguientes cuatro conjuntos de datos:

X_entrenamiento: Los valores de las características que utilizaremos para entrenar el modelo<br>
y_train: Las etiquetas correspondientes que utilizaremos para entrenar el modelo.<br>
X_prueba: Los valores de las características que utilizaremos para validar el modelo<br>
y_test: Las etiquetas correspondientes que utilizaremos para validar el modelo.<br>
Ahora estamos listos para entrenar un modelo ajustando un algoritmo de regresión adecuado a los datos de entrenamiento.

Experimentar con algoritmos

El algoritmo de regresión lineal que utilizamos la última vez para entrenar el modelo tiene cierta capacidad predictiva, pero hay muchos tipos de algoritmo de regresión que podríamos probar, entre ellos:

Algoritmos lineales: No sólo el algoritmo de regresión lineal que utilizamos anteriormente (que técnicamente es un algoritmo de mínimos cuadrados ordinarios), sino otras variantes como Lasso y Ridge.<br>
Algoritmos basados en árboles: Algoritmos que construyen un árbol de decisión para llegar a una predicción.<br>
Algoritmos de conjunto: Algoritmos que combinan los resultados de múltiples algoritmos base para mejorar la generalizabilidad.

Pruebe otro algoritmo lineal
Intentemos entrenar nuestro modelo de regresión utilizando un algoritmo Lasso. Podemos hacerlo simplemente cambiando el estimador en el código de entrenamiento.

In [ ]:
from sklearn.linear_model import Lasso

# Fit a lasso model on the training set
model = Lasso().fit(X_train, y_train)
print (model, "\n")

# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Daily Bike Share Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

Pruebe un algoritmo de árbol de decisión

Como alternativa a un modelo lineal, existe una categoría de algoritmos para el aprendizaje automático que utiliza un enfoque basado en árboles en el que las características del conjunto de datos se examinan en una serie de evaluaciones, cada una de las cuales da lugar a una rama en un árbol de decisión basado en el valor de la característica. Al final de cada serie de ramas se encuentran los nodos hoja con el valor de la etiqueta predicha en función de los valores de las características.

Es más fácil ver cómo funciona esto con un ejemplo. Vamos a entrenar un modelo de regresión de árbol de decisión utilizando los datos de alquiler de bicicletas. Después de entrenar el modelo, el código siguiente imprimirá la definición del modelo y una representación de texto del árbol que utiliza para predecir los valores de las etiquetas.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text

# Train the model
model = DecisionTreeRegressor().fit(X_train, y_train)
print (model, "\n")

# Visualize the model tree
tree = export_text(model)
print(tree)

Ya tenemos un modelo basado en árboles, pero ¿es bueno? Evaluémoslo con los datos de prueba.

In [ ]:
# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Daily Bike Share Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

El modelo basado en árboles no parece haber mejorado con respecto al modelo lineal, así que ¿qué más podríamos probar?

Probar un algoritmo de conjunto
Los algoritmos de conjunto combinan varios estimadores de base para producir un modelo óptimo, ya sea aplicando una función agregada a una colección de modelos de base (lo que a veces se denomina bagging) o construyendo una secuencia de modelos que se basan unos en otros para mejorar el rendimiento predictivo (lo que se denomina boosting).

Por ejemplo, probemos un modelo de Bosque Aleatorio, que aplica una función de promediado a múltiples modelos de Árbol de Decisión para obtener un modelo global mejor.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Train the model
model = RandomForestRegressor().fit(X_train, y_train)
print (model, "\n")

# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Daily Bike Share Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

Por si fuera poco, probemos también un algoritmo de conjunto de refuerzo. Utilizaremos un estimador Gradient Boosting, que al igual que un algoritmo Random Forest construye múltiples árboles, pero en lugar de construirlos todos independientemente y tomar el resultado medio, cada árbol se construye sobre los resultados del anterior en un intento de reducir incrementalmente la pérdida (error) en el modelo.

In [ ]:
# Train the model
from sklearn.ensemble import GradientBoostingRegressor

# Fit a lasso model on the training set
model = GradientBoostingRegressor().fit(X_train, y_train)
print (model, "\n")

# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Daily Bike Share Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

Resumen

Aquí hemos probado una serie de nuevos algoritmos de regresión para mejorar el rendimiento. En nuestro cuaderno veremos cómo "afinar" estos algoritmos para mejorar el rendimiento.

Lecturas adicionales

Para obtener más información sobre Scikit-Learn, consulte la documentación de Scikit-Learn https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics